In [65]:
# Data: http://share.mailcharts.com/0P092V2m0V2x
# Headers: id, company name, company id, subject, sent at, discounts

# Goal: Will the next email be a discount?
# -- True || False
# Goal 2: How much will they discount by?

In [134]:
from datetime import date, datetime
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [146]:
df = pd.read_csv('./data/capstone.csv')
df.head()

,id,company_name,company_id,subject,sent_at,promotion_type,promotion_value
0,1276718,Gem Shopping Network,5929,Happy New Year - Bling in the new year with us!,2017-01-01 00:00:43,[null],[null]
1,1276721,FramesDirect.com,3527,Final Hours - Don't Lose Flex Spending Dollars...,2017-01-01 00:01:32,"[""discount_absolute"", ""discount_percentage"", ""...","[50, 25, 50]"
2,1276725,REVOLVE,2329,"Last day of 2016, LAST DAY OF THE SALE",2017-01-01 00:01:32,[null],[null]
3,1276724,REVOLVE,4490,"Last day of 2016, LAST DAY OF THE SALE",2017-01-01 00:01:44,[null],[null]
4,1276726,The RealReal,3515,40% Off New Jewelry & Watch Markdowns Starts Now,2017-01-01 00:04:04,"[""discount_percentage"", ""discount_percentage"",...","[70, 40, 20]"


In [136]:
# Clean the data

def determine_weekday(n):
    if n == 5 or n == 6:
        # weekend
        return 0
    else:
        #weekday
        return 1
    
def determine_am_pm(date):
    return 0 if date.strftime('%p') == "AM" else 1
    
def create_datetime(row):
    return datetime(row["year"],row["month"],row["day"])

df["promo_not_promo"] = np.where(df["promotion_type"] == "[null]", "0", "1")
df["month"] = df["sent_at"].str.split('-').str.get(1).astype(int)
df["year"] = df["sent_at"].str.split('-').str.get(0).astype(int)
df["day"] = df["sent_at"].str.split('-').str.get(2).str.split(' ').str.get(0).astype(int)
df["weeknum"] = df.apply(lambda x: create_datetime(x).isocalendar()[1], axis=1)
df["am_pm"] = df.apply(lambda x: determine_am_pm(create_datetime(x)), axis=1)
df["weekday_weekend"] = df.apply(lambda x: determine_weekday(create_datetime(x).weekday()), axis=1)
df.head()

,id,company_name,company_id,subject,sent_at,promotion_type,promotion_value,promo_not_promo,month,year,day,weeknum,am_pm,weekday_weekend
0,1276718,Gem Shopping Network,5929,Happy New Year - Bling in the new year with us!,2017-01-01 00:00:43,[null],[null],0,1,2017,1,52,0,0
1,1276721,FramesDirect.com,3527,Final Hours - Don't Lose Flex Spending Dollars...,2017-01-01 00:01:32,"[""discount_absolute"", ""discount_percentage"", ""...","[50, 25, 50]",1,1,2017,1,52,0,0
2,1276725,REVOLVE,2329,"Last day of 2016, LAST DAY OF THE SALE",2017-01-01 00:01:32,[null],[null],0,1,2017,1,52,0,0
3,1276724,REVOLVE,4490,"Last day of 2016, LAST DAY OF THE SALE",2017-01-01 00:01:44,[null],[null],0,1,2017,1,52,0,0
4,1276726,The RealReal,3515,40% Off New Jewelry & Watch Markdowns Starts Now,2017-01-01 00:04:04,"[""discount_percentage"", ""discount_percentage"",...","[70, 40, 20]",1,1,2017,1,52,0,0


In [137]:
dummy_company = pd.get_dummies(df["company_name"])
dummy_company.head()

,Abercrombie & Fitch,Ace & Everett,Active Ride Shop,Adorama,Altrec.com,Amazon.co.uk,Ames Walker,Andrew Christian,Ann Taylor,Annie Selke,...,Thomas Pink,Timothy's Café,Tommy Bahama,Treatsie,Variety,White Stuff,Wine Library,WoolOvers,ZALORA,boohoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
df = df.sort_values("sent_at", ascending=False)
results = []
for index, row in df.iterrows():
    company_emails = df[df["company_name"] == row["company_name"]]
    if np.where(company_emails["sent_at"] < row["sent_at"])[0] > 0:
        for i, r in company_emails.iterrows():
            if r["sent_at"] < row["sent_at"]:
                results.append(r["sent_at"])
                continue
    else:
        results.append(None)

In [143]:
# Homework: Get time delta, merge everything into on DF, create a test dataset with 10,000

In [149]:
datetime(2017,1,1,1,4)

datetime.datetime(2017, 1, 1, 1, 4)